In [6]:
import os
import numpy as np
import pandas as pd

In [7]:
year_list = ['2019', '2020', '2021', '2022', '2023']

##以下代码只跑一次

In [8]:
url = ["Data/listings-2019.csv",
       "Data/listings-2020.csv",
       "Data/listings-2021.csv",
       "Data/listings-2022.csv",
       "Data/listings-2023.csv"]

In [9]:
cols = ['price', 'room_type', 'neighbourhood_cleansed', 'review_scores_rating']
cols_2019 = ['price', 'neighbourhood'] #only 2019

In [10]:
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")
    if year_list[year] == '2019':
        df = df[cols_2019]
        df['year'] = year_list[year]
    else:
        df = df[cols]
        df['year'] = year_list[year]
    
    path = os.path.join('Data','raw') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "drop_cols_" + fn 
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    df.to_csv(os.path.join(path,fn), index=False)
    print(f"{year} Done.")

Data frame is 85,236 x 18
Writing to: drop_cols_listings-2019.csv
0 Done.
Data frame is 76,984 x 74
Writing to: drop_cols_listings-2020.csv
1 Done.
Data frame is 70,617 x 74
Writing to: drop_cols_listings-2021.csv
2 Done.
Data frame is 69,351 x 75
Writing to: drop_cols_listings-2022.csv
3 Done.
Data frame is 87,946 x 75
Writing to: drop_cols_listings-2023.csv
4 Done.


##以上代码只跑一次

In [11]:
url = ["Data/raw/drop_cols_listings-2019.csv",
       "Data/raw/drop_cols_listings-2020.csv",
       "Data/raw/drop_cols_listings-2021.csv",
       "Data/raw/drop_cols_listings-2022.csv",
       "Data/raw/drop_cols_listings-2023.csv"]

In [16]:
for year in range(0, len(year_list)):
    df = pd.read_csv(url[year], low_memory=False) # compression='gzip'
    print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

    if year_list[year] == '2019':
        df.drop(df[df['price'] == ' '].index, inplace=True)
        df['price'] = df['price'].astype('float')
        df_new = df.groupby('neighbourhood').agg({'neighbourhood': 'count', 'price': 'mean'}).rename(columns={'neighbourhood': 'region'})
        df_new.reset_index(inplace=True)
        df_new.columns = ['region', 'count_airbnb', 'price_average']
        df['year'] = year_list[year]
    else:   
        df['price'] = df['price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')
        df_new = df.groupby('neighbourhood_cleansed').agg({'neighbourhood_cleansed': 'count', 'price': 'mean', 'review_scores_rating': 'mean'}).rename(columns={'neighbourhood_cleansed': 'region'})
        df_new.reset_index(inplace=True)
        df_new.columns = ['region', 'count', 'price_average', 'review_scores_rating_average']
        df['year'] = year_list[year]
    
    print(df_new.info())
    print(f"Data frame is {df_new.shape[0]:,} x {df_new.shape[1]}")
    
    path = os.path.join('Data','clean') # A default location to save raw data
    fn = url[year].split('/')[-1]
    fn = "Cleaning_done_" + fn.replace("drop_cols_", "")
    print(f"Writing to: {fn}")
    if not os.path.exists(path):
        print(f"Creating {path} under {os.getcwd()}")
        os.makedirs(path)
    df_new.to_csv(os.path.join(path,fn), index=False)
    print(f"Clean {year} Done.")

Data frame is 85,236 x 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   region         33 non-null     object 
 1   count_airbnb   33 non-null     int64  
 2   price_average  33 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 924.0+ bytes
None
Data frame is 33 x 3
Writing to: Cleaning_done_listings-2019.csv
Clean 0 Done.
Data frame is 76,984 x 5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   region                        33 non-null     object 
 1   count                         33 non-null     int64  
 2   price_average                 33 non-null     float64
 3   review_scores_rating_average  33 non-null     float64
dtypes: float64(2), i